# ByteTrack

In [ ]:
import pickle
import numpy as np
import json
import torch
from tqdm import tqdm
from dataclasses import dataclass
np.float = float

# Import ByteTrack from YOLOX
from yolox.tracker.byte_tracker import BYTETracker, STrack

#===================
# ByteTrack Settings
#===================
@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.9
    track_buffer: int = 10
    match_thresh: float = 0.48
    aspect_ratio_thresh: float = 3
    min_box_area: float = 1
    mot20: bool = False

#========================
# Helper Function: Load 3D Boxes
#========================
def load_bounding_boxes(file_path):
    """Load 3D boxes from a Pickle file."""
    with open(file_path, 'rb') as f:
        box_list = pickle.load(f)
    return box_list

#=============================
# Helper Function: 3D to 2D Box
#=============================
def convert_3d_box_to_2d_bbox(box):
    """Convert a 3D box to a 2D box by dropping rotation."""
    center_x = box[0]
    center_y = box[1]
    length = box[3]
    width  = box[4]

    x1 = center_x - length / 2.0
    y1 = center_y - width  / 2.0
    x2 = center_x + length / 2.0
    y2 = center_y + width  / 2.0

    return [x1, y1, x2, y2]

#===============================
# Helper Function: Box with Track ID
#===============================
def create_box_result(box, track_id):
    """Create a dictionary with 3D box info and track ID."""
    center = np.array(box[:3])     # (x, y, z)
    extent = np.array(box[3:6])    # (length, width, height)
    yaw    = float(box[6])

    return {
        'uuid': str(track_id),
        'label': 'Car',
        'position': {
            'x': float(center[0]),
            'y': float(center[1]),
            'z': float(center[2])
        },
        'dimensions': {
            'x': float(extent[1]),  # width
            'y': float(extent[0]),  # length
            'z': float(extent[2])   # height
        },
        'yaw': yaw
    }

#======================
# Helper Function: IoU
#======================
def compute_iou(boxA, boxB):
    """Compute Intersection over Union (IoU) between two 2D boxes."""
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH

    areaA = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    areaB = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    return interArea / float(areaA + areaB - interArea + 1e-6)

#===========================
# Helper Function: Box Center
#===========================
def center_of_bbox_2d(bbox2d):
    """Return the center (x, y) of a 2D bounding box."""
    x_center = (bbox2d[0] + bbox2d[2]) / 2.0
    y_center = (bbox2d[1] + bbox2d[3]) / 2.0
    return x_center, y_center

#==============================
# Helper Function: 2D Distance
#==============================
def distance_2d(p1, p2):
    """Euclidean distance between two 2D points."""
    return np.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

#==========================================
# Helper Function: Convert Numpy to Python
#==========================================
def convert_to_python_types(data):
    """Convert NumPy types to native Python types for JSON saving."""
    if isinstance(data, dict):
        return {key: convert_to_python_types(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [convert_to_python_types(item) for item in data]
    elif isinstance(data, np.ndarray):
        return data.tolist()
    elif isinstance(data, (np.float32, np.float64)):
        return float(data)
    elif isinstance(data, (np.int32, np.int64)):
        return int(data)
    return data

#====================================================
# Class: ByteTrack Tracker with Speed-Aware Expansion
#====================================================
class SpeedAwareByteTracker:
    def __init__(self, config=None):
        # Init ByteTracker
        self.bytetracker = BYTETracker(BYTETrackerArgs())
        
        # Store previous centers for speed estimation
        self.prev_centers = None

        # Expansion factor for wider boxes (based on motion)
        self.expansion_ratio = config.get('expansion_ratio', 0.2) if config else 0.2

    def visualize_bounding_boxes(self, frames):
        """
        Run ByteTrack on sequence of 3D box frames.
        Returns list of per-frame tracking results.
        """
        frame_results = []
        self.prev_centers = None  # No previous frame at the start

        for frame_idx, frame_boxes in enumerate(tqdm(frames, desc="Tracking frames")):
            raw_bboxes_2d = [convert_3d_box_to_2d_bbox(bx) for bx in frame_boxes]

            # If there's a previous frame, estimate motion and expand box
            if self.prev_centers is not None and len(self.prev_centers) > 0:
                adjusted_bboxes_2d = []
                used_indices_prev = set()

                for new_idx, bbox2d in enumerate(raw_bboxes_2d):
                    c_new = center_of_bbox_2d(bbox2d)
                    best_dist = float('inf')
                    best_prev_idx = None
                    for old_idx, c_old in self.prev_centers.items():
                        d = distance_2d(c_new, c_old)
                        if d < best_dist:
                            best_dist = d
                            best_prev_idx = old_idx

                    if best_prev_idx is not None:
                        c_old = self.prev_centers[best_prev_idx]
                        vx = c_new[0] - c_old[0]
                        vy = c_new[1] - c_old[1]

                        # Move center slightly using speed
                        x1, y1, x2, y2 = bbox2d
                        w = (x2 - x1)
                        h = (y2 - y1)
                        cx = (x1 + x2) / 2.0
                        cy = (y1 + y2) / 2.0

                        alpha = 0.05
                        cx_pred = cx + alpha * vx
                        cy_pred = cy + alpha * vy

                        x1_new = cx_pred - w/2
                        y1_new = cy_pred - h/2
                        x2_new = cx_pred + w/2
                        y2_new = cy_pred + h/2

                        # Expand box by ratio
                        dw = w * self.expansion_ratio / 2
                        dh = h * self.expansion_ratio / 2
                        x1_new -= dw
                        y1_new -= dh
                        x2_new += dw
                        y2_new += dh

                        adjusted_bboxes_2d.append([x1_new, y1_new, x2_new, y2_new, 1.0, 1.0])
                    else:
                        # No match in previous frame
                        x1, y1, x2, y2 = bbox2d
                        w = (x2 - x1)
                        h = (y2 - y1)
                        dw = w * self.expansion_ratio / 2
                        dh = h * self.expansion_ratio / 2
                        adjusted_bboxes_2d.append([x1 - dw, y1 - dh, x2 + dw, y2 + dh, 1.0, 1.0])

                detections = np.array(adjusted_bboxes_2d, dtype=np.float32)
            else:
                # First frame — apply default expansion
                adjusted_bboxes_2d = []
                for bbox2d in raw_bboxes_2d:
                    x1, y1, x2, y2 = bbox2d
                    w = (x2 - x1)
                    h = (y2 - y1)
                    dw = w * self.expansion_ratio / 2
                    dh = h * self.expansion_ratio / 2
                    adjusted_bboxes_2d.append([x1 - dw, y1 - dh, x2 + dw, y2 + dh, 1.0, 1.0])
                detections = np.array(adjusted_bboxes_2d, dtype=np.float32)

            # Run ByteTrack
            if len(detections) == 0:
                detections = np.empty((0, 6), dtype=np.float32)

            detections_tensor = torch.from_numpy(detections)
            img_info = [1000, 1000]  # dummy image size
            img_size = (1000, 1000)
            tracks = self.bytetracker.update(detections_tensor, img_info, img_size)

            frame_cuboids = []
            assigned_detection_idx = set()

            for track in tracks:
                track_bbox = track.tlbr
                best_iou = 0
                best_det_idx = -1
                for det_idx, det in enumerate(detections):
                    if det_idx in assigned_detection_idx:
                        continue
                    iou_val = compute_iou(track_bbox, det[:4])
                    if iou_val > best_iou:
                        best_iou = iou_val
                        best_det_idx = det_idx

                if best_det_idx >= 0 and best_iou > 0.3:
                    assigned_detection_idx.add(best_det_idx)
                    if best_det_idx < len(frame_boxes):
                        original_3d_box = frame_boxes[best_det_idx]
                    else:
                        continue

                    box_result = create_box_result(original_3d_box, track.track_id)
                    frame_cuboids.append(box_result)

            # Save current centers for next frame
            new_centers = {}
            for idx, det in enumerate(detections):
                cx, cy = center_of_bbox_2d(det[:4])
                new_centers[idx] = (cx, cy)
            self.prev_centers = new_centers

            frame_results.append({'cuboids': frame_cuboids})

        return frame_results

#===================
# Main Script
#===================
if __name__ == "__main__":
    # Load 3D boxes
    box_list = load_bounding_boxes("box_list(kopia).pkl")

    # Define tracker config
    config = {
        'expansion_ratio': 25.1  # control how much to expand the boxes
    }

    # Initialize tracker
    tracker = SpeedAwareByteTracker(config)

    # Run tracking
    frame_results = tracker.visualize_bounding_boxes(box_list)

    # Convert and save results
    converted_results = convert_to_python_types(frame_results)
    output_path = "bytetrack5_final2.json"  # fixed typo: was "utput_path"
    with open(output_path, 'w') as json_file:
        json.dump(converted_results, json_file, indent=4)

    print(f"Done! Results saved in {output_path}")
